In [40]:
import json

## 1. Collect Table data

In [41]:
table_dataset_path = "spider/tables.json"
table_dict = {} # table_dict[db_id] = linearized_table

data = json.load(open(table_dataset_path))
for example in data:
    outputs = []
    db_id = example["db_id"]
    outputs.append(db_id)
    
    orig_columns = example["column_names_original"] # referred paper use original column names and table names
    orig_table_names = example["table_names_original"]
    primary_keys = example["primary_keys"]
    
    prev_table_id = -1
    table_row = ""
    for i, column in enumerate(orig_columns):
        table_id, column_name = column[0], column[1]
        if table_id == -1:
            continue
        if table_id != prev_table_id:
            if table_row:
                outputs.append(table_row.strip(", "))
            table_row = "{} : ".format(orig_table_names[table_id])
            prev_table_id = table_id
            
        if i in primary_keys:
            table_row += "{} ( Kyle ), ".format(column_name)
        else:
            table_row += "{}, ".format(column_name)
            
    outputs.append(table_row.strip(", "))
    # ["network_1", "highschooler : id, name ( Kyle ), grade", "friend : student_id, friend_id", ...]
    table_dict[db_id] = outputs 
    
table_dict["network_1"]

['network_1',
 'Highschooler : ID ( Kyle ), name, grade',
 'Friend : student_id ( Kyle ), friend_id',
 'Likes : student_id ( Kyle ), liked_id']

## 2. Prepare Spider input dataset

In [44]:
train_dataset_path = "spider/train_spider.json"
dev_dataset_path = "spider/dev.json"

def prepare_spider_dataset(dataset_path, table_dict):
    input_train_dataset = []
    input_train_dataset_path = "codeT5_" + dataset_path.split("/")[-1]

    data = json.load(open(dataset_path))
    for example in data:
        instance = {}
        db_id = example["db_id"]
        question = example["question"]
        
        inputs = [question] + table_dict[db_id]
        input_str = " | ".join(inputs)
        label = example["query"]
        
        instance["intent"] = input_str
        instance["snippet"] = label
        input_train_dataset.append(instance)
    print(len(input_train_dataset))
    # write list to .json file
    json.dump(input_train_dataset, open("data/" + input_train_dataset_path, "w"), indent=4)


prepare_spider_dataset(train_dataset_path, table_dict)
prepare_spider_dataset(dev_dataset_path, table_dict)
    

7000
1034
